In [1]:
import utils

In [2]:
labels="'earthquake', 'quake', 'temblor'"
time=True
participants=False
location=True
strictLocations=True
pageRankFilter=True
pr_limit=500.0

In [3]:
query=utils.get_me_a_query(labels, time, location, participants, strictLocations, pageRankFilter, pr_limit)
print(query)


	PREFIX sem: <http://semanticweb.cs.vu.nl/2009/11/sem/>
	PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
	PREFIX dbpedia: <http://dbpedia.org/resource/>
	PREFIX owltime: <http://www.w3.org/TR/owl-time#>
	PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
	PREFIX dbo: <http://dbpedia.org/ontology/>
	PREFIX vrank: <http://purl.org/voc/vrank#>
	SELECT ?event ?location ?time ?participant WHERE {
    
		?event a sem:Event .
		{ ?event rdfs:label ?label .
		FILTER (?label IN ('earthquake', 'quake', 'temblor')) } 
		UNION
		{ ?event sem:hasActor ?nonentity .
		?nonentity skos:relatedMatch dbpedia:Earthquake } . 
		?event sem:hasTime ?tmx . ?tmx owltime:inDateTime ?time .  
		?event sem:hasPlace ?location .  
		?location a ?x . ?x rdfs:subClassOf* dbo:Place . 
		FILTER (!REGEX(STR(?x), 'http://dbpedia.org/ontology/Country')) .
		FILTER NOT EXISTS { ?x rdfs:subClassOf* dbo:Country  } .   
		?location vrank:hasRank/vrank:rankValue ?pagerank .
		FILTER (xsd:float(?pagerank)<=500.000000) .


In [4]:
res=utils.get_sparql_results(query)
len(res)

84

In [5]:
from collections import Counter
from collections import defaultdict
import operator

In [6]:
res[0]

{'event': {'type': 'uri',
  'value': 'http://www.newsreader-project.eu/data/wikinews/sigmed/data/d7ddc614-701e-4700-917a-0efc25aa3459.in.naf#ev59'},
 'location': {'type': 'uri',
  'value': 'http://dbpedia.org/resource/Banda_Aceh'},
 'time': {'type': 'uri',
  'value': 'http://www.newsreader-project.eu/time/2004'}}

In [7]:
results = defaultdict(set)
for item in res:
    key = (item['location']['value'], item['time']['value'])
    results[key].add(item['event']['value'])

In [8]:
counter = 0
for key, value in results.items():
    if len(value) >= 2:
        print()
        print(key)
        print(value)


('http://dbpedia.org/resource/Fraser_Island', 'http://www.newsreader-project.eu/time/20150801')
{'http://www.newsreader-project.eu/data/wikinews/sigmed/data/ea04ca8e-c774-42ac-9b1f-7d6a4caf5433.in.naf#ev2', 'http://www.newsreader-project.eu/data/wikinews/sigmed/data/ee3ce871-8cb6-4d0d-9fd0-4cf9b575488f.in.naf#ev2'}

('http://dbpedia.org/resource/Cushing,_Oklahoma', 'http://www.newsreader-project.eu/time/20150918')
{'http://www.newsreader-project.eu/data/wikinews/sigmed/data/5c526c8a-7af1-4869-a78b-6bd1f253cbbd.in.naf#ev150', 'http://www.newsreader-project.eu/data/wikinews/sigmed/data/9aa009a7-efa8-43ba-bd5f-e4d4e751cd45.in.naf#ev145', 'http://www.newsreader-project.eu/data/wikinews/sigmed/data/80242441-59ba-44a4-844b-b92755318585.in.naf#ev150'}

('http://dbpedia.org/resource/Illapel', 'http://www.newsreader-project.eu/time/20150917')
{'http://www.newsreader-project.eu/data/wikinews/sigmed/data/524526ce-6ae7-4593-a272-219ad20a8e78.in.naf#ev9', 'http://www.newsreader-project.eu/data/wik